In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from collections import Counter

In [2]:
data = pd.read_csv(r"C:\Users\asus\Desktop\RecSys\total_features.csv")

In [ ]:
data = data.sort_values(by='timestamp')

# Разделяем данные на train и test выборки
train_size = 0.7
train, test = train_test_split(data, train_size=train_size, shuffle=False)

# Выводим размеры выборок
print(f'Train size: {len(train)}')
print(f'Test size: {len(test)}')

In [ ]:
data.head(3)

Сделаем коллаборативную фильтрацию гибридным методом (как item- так и user- based)

In [12]:
def hybrid_filtration(interaction_matrix_scaled, user_similarity_df, item_similarity_df,  client, num_recommendations):
    interacted_items = interaction_matrix_scaled.loc[client][interaction_matrix_scaled.loc[client] > 0].index.tolist()
    # Функция для получения рекомендаций на основе User-based CF
    def get_user_based_recommendations(user_id, num_recommendations=num_recommendations):
        similar_users = user_similarity_df[user_id].sort_values(ascending=False).index[1:num_recommendations + 1]
        recommended_items = interaction_matrix_scaled.loc[similar_users].sum().sort_values(ascending=False).index[:num_recommendations]
        return recommended_items

    # Функция для получения рекомендаций на основе Item-based CF
    def get_item_based_recommendations(item_id, num_recommendations=num_recommendations*3):
        similar_items = item_similarity_df[item_id].sort_values(ascending=False).index[1:num_recommendations + 1]
        return similar_items

    def get_all_item_recomendations(user_id, num_recommendations=num_recommendations):
        item_based_recommendations = []
        for item_id in interacted_items:
            item_based_recommendations.extend(get_item_based_recommendations(item_id, num_recommendations * 3))
        filtered_based_recommendations = [item for item in item_based_recommendations if item not in interacted_items]
        recommendation_counts = Counter(filtered_based_recommendations)
        # Получаем топ num_recommendations
        top_recommendations = recommendation_counts.most_common(num_recommendations)
        # Возвращаем только предметы
        return [item for item, count in top_recommendations]


    # Гибридная рекомендация
    def hybrid_recommendation(user_id, num_recommendations=num_recommendations):
        user_based_recommendations = get_user_based_recommendations(user_id, num_recommendations)
        user_filtered_based_recommendations = [item for item in user_based_recommendations if item not in interacted_items]
        item_based_recommendations = get_all_item_recomendations(user_id, num_recommendations)
        # Объединение рекомендаций
        # print(user_based_recommendations)
        # print(item_based_recommendations)
        combined_recommendations = set(user_based_recommendations).union(set(item_based_recommendations))
        return tuple([user_filtered_based_recommendations, item_based_recommendations])

    # Пример использования
    return  hybrid_recommendation(client)


In [13]:
from tqdm import tqdm
def get_filter_recs(data):
    
    # Создание матрицы взаимодействий
    interaction_matrix = data.pivot(index='user_id', columns='item_id', values='timestamp').fillna(0)

    # Нормализация матрицы взаимодействий
    # scaler = StandardScaler()
    # interaction_matrix_scaled = scaler.fit_transform(interaction_matrix)

    # Коллаборативная фильтрация (User-based)
    user_similarity = cosine_similarity(interaction_matrix)
    user_similarity_df = pd.DataFrame(user_similarity, index=interaction_matrix.index, columns=interaction_matrix.index)

    # Коллаборативная фильтрация (Item-based)
    item_similarity = cosine_similarity(interaction_matrix.T)
    item_similarity_df = pd.DataFrame(item_similarity, index=interaction_matrix.columns, columns=interaction_matrix.columns)
    recommendation_dict ={}
    for user in tqdm(data.user_id.unique()):
        recommendation_dict[user] = hybrid_filtration(interaction_matrix, user_similarity_df, item_similarity_df,  user, num_recommendations = 50 )
    return recommendation_dict
    

Получили словарь рекомендаций (как item- так и user- based)

In [15]:
recommendation_dict = get_filter_recs(train)

100%|██████████| 6040/6040 [12:24<00:00,  8.11it/s]


Получим train-датасет для обучения бустинга - выберем пары, по которым не было взаимодействий

In [16]:
unique_users = data['user_id'].unique()
unique_items = data['item_id'].unique()

# Создаем все возможные пары user_id и item_id
all_pairs = pd.MultiIndex.from_product([unique_users, unique_items], names=['user_id', 'item_id']).to_frame(index=False)

# Находим пары, которые есть в train
train_pairs = train[['user_id', 'item_id']]

# Удаляем пары, которые есть в train из all_pairs
val = all_pairs.merge(train_pairs, on=['user_id', 'item_id'], how='left', indicator=True)
val = val[val['_merge'] == 'left_only'].drop(columns=['_merge'])

In [17]:
def add_recommendation_columns(data, recommendation_dict):
    """Add columns indicating if there are recommendations for each algorithm."""
    recommended_algo1 = []
    recommended_algo2 = []
    for index, row in tqdm(data.iterrows()):
        user = row['user_id']
        item = row['item_id']
        if user in recommendation_dict.keys():
            if item in recommendation_dict[user][0]:
                recommended_algo1.append(1)
            else:
                recommended_algo1.append(0)
            if item in recommendation_dict[user][1]:
                recommended_algo2.append(1)
            else:
                recommended_algo2.append(0)
        else:
            recommended_algo1.append(0)
            recommended_algo2.append(0)
    
    return recommended_algo1, recommended_algo2

Добавим рекомендации из коллаборативной фильтрации

In [18]:
val['recommended_algo1'], val['recommended_algo2'] = add_recommendation_columns(val, recommendation_dict)

21661696it [25:30, 14152.84it/s]


In [19]:
rating_dict = test.set_index(['user_id', 'item_id'])['rating'].to_dict()

# Функция для определения значения y
def get_y(row):
    user_id = row['user_id']
    item_id = row['item_id']
    
    # Проверяем, есть ли пара в rating_dict
    if (user_id, item_id) in rating_dict:
        rating = rating_dict[(user_id, item_id)]
        if rating <= 2:
            return 0
        elif rating == 3:
            return 0.5
        elif rating >= 4:
            return 1
    return 0.25  # Если пары нет в test_pairs

# Применяем функцию к каждой строке DataFrame val
val['y'] = val.apply(get_y, axis=1)

In [24]:
def remove_random_share(df, share):
    """
    Удаляет случайную долю строк из DataFrame, где y == 0.25.

    :param df: DataFrame, из которого нужно удалить строки
    :param share: Доля строк для удаления (например, 0.1 для 10%)
    :return: Очищенный DataFrame
    """
    # Фильтруем строки, где y == 0.25
    rows_to_remove = df.query('y ==0.25 and recommended_algo1 ==0 and recommended_algo2 ==0')
    
    # Определяем количество строк для удаления
    num_rows_to_remove = int(len(rows_to_remove) * share)
    
    # Проверяем, что количество строк для удаления не превышает доступное количество
    if num_rows_to_remove > 0:
        # Случайным образом выбираем строки для удаления
        rows_to_remove_indices = rows_to_remove.sample(n=num_rows_to_remove, random_state=42).index
        
        # Удаляем выбранные строки из оригинального DataFrame
        df_cleaned = df.drop(index=rows_to_remove_indices)
    else:
        # Если нет строк для удаления, возвращаем оригинальный DataFrame
        df_cleaned = df.copy()
    
    return df_cleaned

Нам не хватает оперативки - придётся обучить catboost  на обрезанном датасете. Мы оставили все пары, которые есть в test, и по котрым есть рекомендации из коллаборативной фильтрации

In [25]:
val_small = remove_random_share(val, 0.90)

In [26]:
len(val_small)

2716381

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


Добавим фичи по клиента и itemам

In [28]:
client_features = data.groupby(by = 'user_id')['age_group', 'user_rating_count', 'user_genre0_share', 'user_genre1_share',
    'user_genre2_share', 'user_genre3_share', 'user_genre4_share',
    'user_genre5_share', 'user_genre6_share', 'user_genre7_share',
    'user_genre8_share', 'user_genre9_share', 'user_genre10_share',
    'user_genre11_share', 'user_genre12_share', 'user_genre13_share',
    'user_genre14_share', 'user_genre15_share', 'user_genre16_share',
    'user_genre17_share', 'user_timestamp_q0', 'user_timestamp_q10',
    'user_timestamp_q25', 'user_timestamp_q33', 'user_timestamp_q50',
    'user_timestamp_q67', 'user_timestamp_q75', 'user_timestamp_q90',
    'user_timestamp_q100', 'user_timestamp_range', 'user_timestamp_iqr',
    'gender', 'age'].mean().reset_index()

C:\Users\asus\AppData\Local\Temp\ipykernel_3860\1232126811.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  client_features = data.groupby(by = 'user_id')['age_group', 'user_rating_count', 'user_genre0_share', 'user_genre1_share',
C:\Users\asus\AppData\Local\Temp\ipykernel_3860\1232126811.py:11: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  'gender', 'age'].mean().reset_index()


In [29]:
item_features = data.groupby(by = 'item_id')['female_ratio', 'male_ratio',
    'young_ratio', 'adult_ratio', 'senior_ratio',
    'female_ratio_genre', 'male_ratio_genre', 'young_ratio_genre',
    'adult_ratio_genre', 'senior_ratio_genre', 'item_rating_count',
    'avg_rating_time_x', 'rating_time_range_x', 
    'genre_0', 'genre_1', 'genre_2', 'genre_3', 'genre_4', 'genre_5',
    'genre_6', 'genre_7', 'genre_8', 'genre_9', 'genre_10', 'genre_11',
    'genre_12', 'genre_13', 'genre_14', 'genre_15', 'genre_16', 'genre_17'].mean().reset_index()

C:\Users\asus\AppData\Local\Temp\ipykernel_3860\3081234336.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  item_features = data.groupby(by = 'item_id')['female_ratio', 'male_ratio',


In [30]:
val_with_client_features = val_small.merge(client_features, on='user_id', how='left')

# Выполняем left join с item_features по item_id
val_final = val_with_client_features.merge(item_features, on='item_id', how='left')

In [31]:
val_final.head(2)

,user_id,item_id,recommended_algo1,recommended_algo2,y,user_rating_count,user_genre0_share,user_genre1_share,user_genre2_share,user_genre3_share,...,genre_8,genre_9,genre_10,genre_11,genre_12,genre_13,genre_14,genre_15,genre_16,genre_17
0,0,1069,0,0,0.25,287.0,0.331,0.2125,0.0906,0.1498,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0,3051,0,0,1.00,287.0,0.331,0.2125,0.0906,0.1498,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
val_final.columns

Index(['user_id', 'item_id', 'recommended_algo1', 'recommended_algo2', 'y',
       'user_rating_count', 'user_genre0_share', 'user_genre1_share',
       'user_genre2_share', 'user_genre3_share', 'user_genre4_share',
       'user_genre5_share', 'user_genre6_share', 'user_genre7_share',
       'user_genre8_share', 'user_genre9_share', 'user_genre10_share',
       'user_genre11_share', 'user_genre12_share', 'user_genre13_share',
       'user_genre14_share', 'user_genre15_share', 'user_genre16_share',
       'user_genre17_share', 'user_timestamp_q0', 'user_timestamp_q10',
       'user_timestamp_q25', 'user_timestamp_q33', 'user_timestamp_q50',
       'user_timestamp_q67', 'user_timestamp_q75', 'user_timestamp_q90',
       'user_timestamp_q100', 'user_timestamp_range', 'user_timestamp_iqr',
       'age', 'female_ratio', 'male_ratio', 'young_ratio', 'adult_ratio',
       'senior_ratio', 'female_ratio_genre', 'male_ratio_genre',
       'young_ratio_genre', 'adult_ratio_genre', 'senior_ratio_g

Обучим бустинг (в этом ноутбуке он упал, мало памяти - будет доп файл, где будет видно, что модель обучена успешно)

In [33]:
import pandas as pd
from catboost import CatBoostRegressor

In [34]:
X = val_final.drop(columns=['y'])  # Все столбцы, кроме 'y'
y = val_final['y']  # Целевая переменная

# Создаем и обучаем модель
model = CatBoostRegressor(iterations=1000,  # Количество итераций
                          learning_rate=0.1,  # Скорость обучения
                          depth=6,  # Глубина деревьев
                          verbose=100,
                          ignored_features = ['user_id', 'item_id'])  # Вывод информации каждые 100 итераций

# Обучение модели
model.fit(X, y)

0:	learn: 0.1645482	total: 2.24s	remaining: 37m 17s
100:	learn: 0.1357963	total: 1m 29s	remaining: 13m 17s
200:	learn: 0.1335464	total: 2m 52s	remaining: 11m 26s
300:	learn: 0.1320875	total: 4m 12s	remaining: 9m 46s
400:	learn: 0.1310442	total: 5m 41s	remaining: 8m 30s
500:	learn: 0.1302476	total: 7m 32s	remaining: 7m 31s


CatBoostError: bad allocation

In [ ]:
joblib.dump(model, 'catboost.joblib')